In [ ]:
# !pip install cdsapi # - if you haven't already

In [52]:
# General libs for file paths, data extraction, etc
from glob import glob
from pathlib import Path
from os.path import basename
import zipfile # To extract zipfiles
import urllib3 
urllib3.disable_warnings() # Disable warnings for data download via API

# CDS API
import cdsapi

In [53]:
import iris
import os
import netCDF4
import numpy
import xarray

**Downloading data**

In [54]:
DATADIR = "/Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august" # directory you want to be working in and writing your files into
os.chdir(DATADIR)
os.getcwd()

'/Users/fionaspuler/Desktop/ECMWF/code_and_data/testing_data_august'

In [ ]:
URL = 'https://cds.climate.copernicus.eu/api/v2'
KEY = '' # enter your key instead

In [ ]:
# choose model - or models, currently not iterated over though
model = ['ipsl_cm6a_lr']
# set variable - not all variables available at daily resolution for cmip6 at the moment
variables = ['near_surface_air_temperature','precipitation',
            'near_surface_wind_speed', 'daily_minimum_near_surface_air_temperature']

In [ ]:
# download historical climate model data

c = cdsapi.Client(url=URL, key=KEY)

for v in variables:
    c.retrieve(
        'projections-cmip6',
        {
            'temporal_resolution': 'daily',
            'experiment': 'historical',
            'level': 'single_levels',
            'variable': v,
            'model': 'ipsl_cm6a_lr',
            'date': '1979-01-01/2015-12-31',
            'area': [
                    52, 8, 48,
                    12,
            ],
            'format': 'zip',
        },
        DATADIR+'/cmip6_daily_1979-2015_ipsl_historical_'+v+'.zip')

In [ ]:
# download future climate model data

c = cdsapi.Client(url=URL, key=KEY)

for v in variables:
    c.retrieve(
        'projections-cmip6',
        {
            'temporal_resolution': 'daily',
            'experiment': 'ssp5_8_5',
            'level': 'single_levels',
            'variable': v,
            'model': 'ipsl_cm6a_lr',
            'date': '2050-01-01/2070-12-31',
            'area': [
                    52, 8, 48,
                    12,
            ],
            'format': 'zip',
        },
        DATADIR+'/cmip6_daily_2050-2070_ipsl_ssp5_8_5_'+v+'.zip')

In [ ]:
# download daily ERA5 data

#https://confluence.ecmwf.int/pages/viewpage.action?pageId=228867588

import cdsapi
import requests
import iris
 
# CDS API script to use CDS service to retrieve daily ERA5* variables and iterate over
# all months in the specified years.
 
# Requires:
# 1) the CDS API to be installed and working on your system
# 2) You have agreed to the ERA5 Licence (via the CDS web page)
# 3) Selection of required variable, daily statistic, etc
 
# Output:
# 1) separate netCDF file for chosen daily statistic/variable for each month
 
c = cdsapi.Client(url=URL, key=KEY, timeout=300)


 
years =  ['1979', '1980', '1981',
            '1982', '1983', '1984',
            '1985', '1986', '1987',
            '1988', '1989', '1990',
            '1991', '1992', '1993',
            '1994', '1995', '1996',
            '1997', '1998', '1999',
            '2000', '2001', '2002',
            '2003', '2004', '2005',
            '2006', '2007', '2008',
            '2009', '2010', '2011',
            '2012', '2013', '2014', '2015']

months = ['01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12']
 
# For valid keywords, see Table 2 of:
# https://datastore.copernicus-climate.eu/documents/app-c3s-daily-era5-statistics/C3S_Application-Documentation_ERA5-daily-statistics-v2.pdf
 
# select your variable; name must be a valid ERA5 CDS API name.
var = "2m_temperature"
 
# Select the required statistic, valid names given in link above
stat = "daily_mean"
 
# Loop over years and months

locations = []

for yr in years:
    for mn in months:
        result = c.service(
        "tool.toolbox.orchestrator.workflow",
        params={
             "realm": "user-apps",
             "project": "app-c3s-daily-era5-statistics",
             "version": "master",
             "kwargs": {
                 "dataset": "reanalysis-era5-single-levels",
                 "product_type": "reanalysis",
                 "variable": var,
                 "statistic": stat,
                 "year": yr,
                 "month": mn,
                 "time_zone": "UTC+00:0",
                 "frequency": "1-hourly",
                 "grid": "1.0/1.0",
                 "area":{"lat": [48, 52], "lon": [8, 12]}
                 },
        "workflow_name": "application"
        })  

 
        file_name = DATADIR+"/era5_" + stat + "_" + var + "_" + yr + "_" + mn + ".nc"
         
        location=result[0]['location']
        res = requests.get(location, stream = True)
        print("Writing data to " + file_name)
        with open(file_name,'wb') as fh:
            for r in res.iter_content(chunk_size = 1024):
                fh.write(r)
        fh.close()

**Process output**

In [55]:
# concatenate all the individual ERA5 files
output_filename = "era5_daily_mean_2m_temperature_1979_2015.nc"

In [ ]:
ds = xarray.open_mfdataset('era5_daily_mean_2m_temperature_*.nc',combine = 'nested', concat_dim="time")
ds.to_netcdf(output_filename)

In [61]:
tas_obs = iris.load_cube(output_filename)
tas_obs

<iris 'Cube' of air_temperature / (K) (time: 13514; latitude: 5; longitude: 5)>

In [57]:
tas_cm_hist_raw = iris.load_cube(DATADIR+'/cmip6_daily_1979-2015_ipsl_historical_near_surface_air_temperature/tas_day_IPSL-CM6A-LR_historical_r1i1p1f1_gr_19790101-20141231_v20190614.nc')
tas_cm_future_raw = iris.load_cube(DATADIR+'/cmip6_daily_2050-2070_ipsl_ssp5_8_5_near_surface_air_temperature/tas_day_IPSL-CM6A-LR_ssp585_r1i1p1f1_gr_20500101-20701231_v20190903.nc')


/Users/fionaspuler/opt/anaconda3/lib/python3.9/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/Users/fionaspuler/opt/anaconda3/lib/python3.9/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [65]:
tas_cm_hist_raw

<iris 'Cube' of air_temperature / (K) (time: 13149; latitude: 4; longitude: 1)>

In [63]:
# regridding 

tas_cm_hist = tas_cm_hist_raw.regrid(tas_obs, iris.analysis.Linear())
tas_cm_future = tas_cm_future_raw.regrid(tas_obs, iris.analysis.Linear())

In [64]:
tas_cm_future

<iris 'Cube' of air_temperature / (K) (time: 7670; latitude: 5; longitude: 5)>